# Cypher query examples to use on the knowledge graph

#### What are the potential gene targets for a given disease?
Since genes and diseases are not directly related in the knowledge graph, you first have to search for the drug that is approved to treat the given disease, and then you can search for the genes that are the targets for that drug. 

In [ ]:
/* Direct relationships */
MATCH (disease:Disease {Name:"OVARIAN CANCER"})<-[diseaseToDrug:APPROVED_TO_TREAT]-(drug:Drug)
WITH disease, diseaseToDrug, drug as d
MATCH (drug:Drug)-[drugToGene]-(gene:Gene)
WHERE drug.Name=d.Name
RETURN DISTINCT disease, diseaseToDrug, d, drug, drugToGene, gene
;

/* Indirect relationships (one further hop, answers the question: What genes are associated with those genes?) */
MATCH (disease:Disease {Name:"OVARIAN CANCER"})<-[diseaseToDrug:APPROVED_TO_TREAT]-(drug:Drug)
WITH disease, diseaseToDrug, drug as d
MATCH (drug:Drug)-[drugToGene]-(gene:Gene)
WHERE drug.Name=d.Name
WITH disease, diseaseToDrug, d, drug, drugToGene, gene as g
MATCH (gene:Gene)-[geneToGene]-(gene2:Gene)
WHERE gene.Symbol=g.Symbol
RETURN DISTINCT disease, diseaseToDrug, d, drug, drugToGene, g, gene, geneToGene, gene2 
;

#### How do two genes potentially interact? 
This question is asking for you to return the relationship between two given genes. 

In [ ]:
/* Direct relationships */
MATCH (g1:Gene {Symbol: "EGFR"})-[geneToGene]-(g2:Gene{Symbol: "TP53"})
RETURN DISTINCT type(geneToGene)
;

/* Indirect relationships (one further hop, if the two genes do not share a direct relationship but are related through one middle gene) */
MATCH (gene:Gene {Symbol: "EGFR"})-[geneToOther]-(othernode)-[otherToGene]-(gene2:Gene {Symbol: "TP53"})
RETURN gene, geneToOther, othernode, otherToGene, gene2 LIMIT 5
;

#### What drugs could be associated with a given gene? 
This question is asking you to return all the drugs that relate to the given gene, it doesn't specify the type of relationship, so you search for that too here.

In [ ]:
MATCH (d:Drug)-[drugToGene]->(g:Gene{Symbol: "BCL2"})
RETURN DISTINCT d, type(drugToGene)
;

#### (1) What drugs have been approved to treat Acute Myeloid Leukemia?

#### (2) What are the targets for these drugs?

#### (3) What are the genes that interact with these target genes?

In [ ]:
/* (1) */
MATCH (drug:Drug)-[drugToDisease:APPROVED_TO_TREAT]->(disease:Disease {Name: "ACUTE MYELOGENOUS LEUKEMIA"})
RETURN DISTINCT drug, drugToDisease, disease

/* (2) */
MATCH (drug:Drug)-[drugToDisease:APPROVED_TO_TREAT]->(disease:Disease {Name: "ACUTE MYELOGENOUS LEUKEMIA"})
WITH disease. drugToDisease, drug as d
MATCH (drug:Drug)-[drugToGene:TARGETS]-(gene:Gene)
WHERE d.Name = drug.Name
RETURN DISTINCT disease, drugToDisease, d, drug, drugToGene, gene

/* (3) */
MATCH (drug:Drug)-[diseaseToDrug:APPROVED_TO_TREAT]->(disease:Disease {Name:"ACUTE MYELOGENOUS LEUKEMIA"})
WITH disease, diseaseToDrug, drug as d
MATCH (drug:Drug)-[drugToGene]-(gene:Gene)
WHERE d.Name = drug.Name 
WITH disease, diseaseToDrug, d, drug, drugToGene, gene as g
MATCH (gene:Gene)-[geneToGene]-(gene2:Gene)
WHERE g.Symbol = gene.Symbol
RETURN DISTINCT disease, diseaseToDrug, d, drug, drugToGene, g, gene, geneToGene, gene2 
;